In [1]:
import os
import glob
import re
import sys
from tkinter import filedialog
from tkinter import *
import fnmatch
import string
import pandas as pd
import xlrd
import datetime

In [2]:
#Please navigate to folder indicating year of measurement:
# e.g. ''\\igswztwwgszona\Gravity Data Archive\Relative Data\TAMA\2018'
root = Tk()
root.withdraw()
folder = filedialog.askdirectory()

In [4]:
#quick check to see all stations included in that year
data = []
newf = str(folder)
for folders in sorted(os.listdir(folder)):
    print(folders)

adtucs
aetucs
af13
av25
c22a
fd62
tucsoncf
wr175


# Define a function to parse each spreadsheet in folders:

In [5]:
def parser(name):
    new = []
    lines = []
    for i in range(name.shape[1]):
        new = []
        one = name.iloc[:,i]
        one = one.dropna()
        one.index=range(len(one))
        #print(one)
        num = one.shape[0]
        
        for i in range(num):
            new1 = one[i]
            #print(type(new1))
            if  isinstance(new1, datetime.date) == True:
                #print('true')
                new1 = '{0.hour}:{0.minute}:{0.second}'.format(new1)
            new.append(new1)

        if new[0] != 'time':
            if new[0] != 'gravity':
                lines.append(new)
    return lines

# Pull out relevant information and make a string for output to .txt file:

In [8]:
data = []
newf = str(folder)
for folders in sorted(os.listdir(folder)):
    for file in sorted(os.listdir(folder + '/' + folders)):
        abs_path = os.path.join(folder + '/' + folders + '/' + file)
        data_xls = pd.read_excel(abs_path,'results', index_col=None, usecols = 7, dtype = 'object')
        yr = data_xls['Unnamed: 3'][0]
        #dat = data_xls['Unnamed: 2'][0]
        #dat = dat.replace(year = yr).date()
        #dat = '{0.year}/{0.month}/{0.day}'.format(dat)
        dat = file[0:4] + '/' + file[4:6] + '/' + file[6:8]
        user = data_xls['Unnamed: 2'][3][0]
        meter1 = data_xls['Unnamed: 2'][6] 
        meter2 = data_xls['Unnamed: 3'][6]
        meter2 = str(meter2)
        meter = meter1 + meter2
        zeros = [0,0,0,0,0,0,0,0,0,0]
        times = data_xls[51:56]
        times.index=range(len(times))
        times = times.dropna(thresh = len(times)-3, axis = 1)

        grav = data_xls[57:62]
        grav.index = range(len(grav))
        grav = grav.dropna(thresh = len(grav) - 3, axis = 1)


        A = parser(times)

        B = parser(grav)
        list = []

        length = (A[0])
        for i in range(len(A)): #for rows in A
            cols = len(A[i])
            for j in range(1,cols): #for columns in A
                #print(j)
                if type(A[i][j]) == datetime.time:
                    A[i][j] = '{0.hour}:{0.minute}:{0.second}'.format(A[i][j])
                new = [A[i][0], user, meter, dat, A[i][j], B[i][j], 0,0,0,0,0,0,0,0,0,0]
                list.append(new)

                #A[0][1] = '{0.hour}:{0.minute}:{0.second}'.format(A[0][1])
                #new = [A[0][0], user, meter, dat, A[0][1], B[0][1], 0,0,0,0,0,0,0,0,0,0]
                #list.append(new)
        


        #convert back to a dataframe because...i only know how to do it this way
        list2 = pd.DataFrame(list)


        # use to_csv but specify .txt to output GSAdjust-readable output file
        list2.to_csv(file +'.txt', sep = ' ', header = None, index = False)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


# Smash together the individual text files just created:

In [9]:
read_files = glob.glob('*.txt')

with open('allTAMA.txt', 'wb') as outfile:
    for f in read_files:
        with open(f, 'rb') as infile:
            outfile.write(infile.read())

# Open up allTAMA.txt, fix it, and rename allTAMA_fixed.txt:

In [10]:
data = pd.read_csv('allTAMA.txt', sep = ' ', header = None)
data.columns = ['sta', 'user', 'meter', 'date','time','gravity','b','c','d','e','f','g','h','i','j','k']

# export to csv and sort by date and then time, fix date format back:

In [11]:
#date format will go wrong in csv, need to force it to be yyyy-mm-dd format before reloading in next cell
data.to_csv('fixdates.csv', sep = ',')

In [22]:
#read it back in, all corrections should have been made now
datanow = pd.read_csv('fixdates.csv')

In [23]:
#look ok?
datanow.head()

,sta,user,meter,date,time,gravity,b,c,d,e,f,g,h,i,j,k
0,david,P,D127,2015/12/15,10:59:00,0.363510,0,0,0,0,0,0,0,0,0,0
1,pond,P,D127,2015/12/15,11:50:00,30.741863,0,0,0,0,0,0,0,0,0,0
2,wr175,P,D127,2015/12/15,12:21:00,68.872224,0,0,0,0,0,0,0,0,0,0
3,guard,P,D127,2015/12/15,12:55:00,11.921801,0,0,0,0,0,0,0,0,0,0
4,wr175,P,D127,2015/12/15,14:18:00,68.874849,0,0,0,0,0,0,0,0,0,0


In [17]:
for i in range(len(datanow)):
    datanow['gravity'][i] = float(datanow['gravity'][i]) * 0.001

C:\Users\ekahler\AppData\Local\Continuum\miniconda3\envs\pyg35\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
datanow.to_csv('TAMAoutput_forGSadjust2016.txt', header = False, index = False, sep = ' ')